#### Test Pipeline
This is a Julia Notebook to check the correctness of the C++ code. 

The notebook proceeds as follows: 
- Generate a synthetic instance
- Run the original implementation of the algorithm (pure Julia)
- Run the C++ implementation of the algorithm

#### Step 1: Data Generation

Generates a ground truth covariance matrix S of the form $I + \beta x_1 x_1^\top + \beta x_2 x_2^\top$, where 
- $x_1, x_2$ are $k$-sparse, with non-overlapping support 
- $\beta$ controls the signal-to-noise ratio

Then, samples $n$ multivariate normal observation from $\mathcal{N}(0,S)$ and constructs the empirical covariance matrix $\Sigma$

In [1]:
using Random, LinearAlgebra
p = 10 #Dimension
r = 2 #Number of sparse PCs
k = 4 #Sparsity of each PC
β = 1 #Signal strength


x1 = zeros(p); x1[1:k] = sign.(rand(k) .- .5)
x2 = zeros(p); x2[(k+1):(k+k)] = sign.(rand(k) .- .5)

# x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap)] = sign.(rand(k_overlap) .- .5)
# x2[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)] = -x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)]
# x2[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)] = x1[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)]

@assert sum(abs.(x1) .> 0) == k
@assert sum(abs.(x2) .> 0) == k
@assert abs(dot(x1,x2)) ≤ 1e-10

shufflecoords = randperm(p)
x1 = x1[shufflecoords]; x2=x2[shufflecoords] 

x1 /= norm(x1); x2 /= norm(x2) 

S = β*x1*x1'+β*x2*x2'+ Matrix(1.0*I, p, p)
S = (S + S')/2

10×10 Matrix{Float64}:
 1.0   0.0   0.0   0.0   0.0   0.0    0.0   0.0   0.0    0.0
 0.0   1.25  0.0   0.0   0.0   0.0   -0.25  0.0   0.25   0.25
 0.0   0.0   1.25  0.25  0.25  0.25   0.0   0.0   0.0    0.0
 0.0   0.0   0.25  1.25  0.25  0.25   0.0   0.0   0.0    0.0
 0.0   0.0   0.25  0.25  1.25  0.25   0.0   0.0   0.0    0.0
 0.0   0.0   0.25  0.25  0.25  1.25   0.0   0.0   0.0    0.0
 0.0  -0.25  0.0   0.0   0.0   0.0    1.25  0.0  -0.25  -0.25
 0.0   0.0   0.0   0.0   0.0   0.0    0.0   1.0   0.0    0.0
 0.0   0.25  0.0   0.0   0.0   0.0   -0.25  0.0   1.25   0.25
 0.0   0.25  0.0   0.0   0.0   0.0   -0.25  0.0   0.25   1.25

In [2]:
n = 1000 #Sample size

Random.seed!(1234)

using Distributions
d = MvNormal(zeros(p), S)
X = rand(d, n) #p by N matrix of observations

Sn = cov(X') #Sample covariance matrix

10×10 Matrix{Float64}:
  0.959347     0.0300463   -0.0275862   …   0.0142444    0.0799508
  0.0300463    1.26725     -0.0761811       0.198116     0.169846
 -0.0275862   -0.0761811    1.28697        -0.00858138   0.0161887
  0.016806    -0.00952819   0.262732       -0.0021708   -0.0163836
 -0.0206788    0.0524347    0.253069        0.0616995   -0.00489492
  0.00260503   0.0252484    0.234962    …  -0.0111205    0.102269
 -0.011102    -0.270139    -0.022575       -0.325806    -0.231496
 -0.00862534  -0.0807322    0.0316437      -0.00298677  -0.0672823
  0.0142444    0.198116    -0.00858138      1.26895      0.242573
  0.0799508    0.169846     0.0161887       0.242573     1.31408

In [3]:
# show(stdout, "text/plain", Sn)

In [4]:
# [x1 x2]

In [5]:
# [k, k]

#### Step 2: Julia benchmark

Applies the Julia code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [8]:
include("algorithm2.jl")

findmultPCs_deflation (generic function with 1 method)

In [9]:
ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(Sn, r, [k,k]; numIters = 20, verbose = true, violation_tolerance = 1e-4 )

x_best

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern: [4, 4]

  Iteration |      Objective value |   Orthogonality Violation |       Time 
          1 |                0.332 |                  2.00e+00 |      0.317 


          2 |                0.332 |                  2.00e+00 |      0.344 
          3 |                0.330 |                  3.33e-16 |      0.379 
          4 |                0.330 |                  4.44e-16 |      0.405 
          5 |                0.330 |                  0.00e+00 |      0.440 


          6 |                0.330 |                  2.22e-16 |      0.474 
          7 |                0.330 |                  3.33e-16 |      0.499 


10×2 Matrix{Float64}:
 0.0        0.0
 0.0        0.448173
 0.519934   0.0
 0.502909   0.0
 0.462533   0.0
 0.512653   0.0
 0.0       -0.562901
 0.0        0.0
 0.0        0.507562
 0.0        0.473987

In [9]:
# show(stdout, "text/plain", x_best)

#### Step 3: R/C++ implementation

Applies the R code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [10]:
using RCall

R"""library(sPCAmPC)"""
# R"""
# library(devtools)
# reload(pkg = "sPCAmPC", quiet = FALSE)"""


RObject{StrSxp}
[1] "sPCAmPC"   "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     


In [11]:
R"""

TestMat <- $Sn 

results <- cpp_findmultPCs_deflation(TestMat, 2, c(4, 4), numIters=20)
"""

In [15]:
R"""
results 
"""

RObject{VecSxp}
$objective_value
[1] 3.956

$orthogonality_violation
[1] 4.440892e-16

$runtime
[1] 0.458

$x_best
           [,1]      [,2]
 [1,] 0.0000000 0.4477586
 [2,] 0.0000000 0.5479709
 [3,] 0.0000000 0.0000000
 [4,] 0.4714282 0.0000000
 [5,] 0.0000000 0.4855453
 [6,] 0.0000000 0.5133088
 [7,] 0.5397896 0.0000000
 [8,] 0.4343905 0.0000000
 [9,] 0.0000000 0.0000000
[10,] 0.5456075 0.0000000



In [16]:
x_best

10×2 Matrix{Float64}:
  0.0       -0.446272
  0.0       -0.549635
  0.0        0.0
 -0.466891   0.0
  0.0       -0.481286
  0.0       -0.516824
 -0.534263   0.0
 -0.433332   0.0
  0.0        0.0
 -0.555697   0.0

In [13]:
[x1 x2]

10×2 Matrix{Float64}:
 -0.5   0.0
  0.0  -0.5
  0.0   0.5
  0.0  -0.5
  0.0   0.0
 -0.5   0.0
  0.0  -0.5
  0.0   0.0
  0.5   0.0
 -0.5   0.0